# IST 718: Big Data Analytics

- Professor: Daniel Acuna <deacuna@syr.edu>

## General instructions:

- You are welcome to discuss the problems with your classmates but __you are not allowed to copy any part of your answers either from your classmates or from the internet__
- You can put the homework files anywhere you want in your http://notebook.acuna.io workspace but _do not change_ the file names. The TAs and the professor use these names to grade your homework.
- Remove or comment out code that contains `raise NotImplementedError`. This is mainly to make the `assert` statement fail if nothing is submitted.
- The tests shown in some cells (i.e., `assert` and `np.testing.` statements) are used to grade your answers. **However, the professor and TAs will use __additional__ test for your answer. Think about cases where your code should run even if it passess all the tests you see.**
- Before downloading and submitting your work through Blackboard, remember to save and press `Validate` (or go to 
`Kernel`$\rightarrow$`Restart and Run All`). 
- Good luck!

In [1]:
# Load the packages needed for this part
# create spark and sparkcontext objects
from pyspark.sql import SparkSession
import numpy as np

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

import pyspark
from pyspark.ml import feature, regression, Pipeline, classification, pipeline, evaluation
from pyspark.sql import functions as fn, Row
from pyspark import sql

import matplotlib.pyplot as plt
import pandas as pd

# Part 2

In this section, you are going to develop a SMS spam detector based on logistic regression. This is the same idea behind sentiment analysis, but instead of predicting positive sentiment vs negative sentiment, you are going to predict whether a SMS text is spam or not.

The dataset will be in `sms_spam_df`

In [2]:
sms_spam_df = spark.read.csv('sms_spam.csv', header=True, inferSchema=True)

# Question 2.1

Encode the `type` column to be 1 for `spam` and 0 for `ham` and store the result in `sms_spam2_df`. Besides, assign the count of spam to `spam_count` and the count of ham to `ham_count`. 

In [3]:
# create sms_spam2_df below

from pyspark.sql.functions import *

sms_spam2_df = sms_spam_df.withColumn('type', 
                                  when(sms_spam_df.type=='spam',1)
                                 .when(sms_spam_df.type=='ham',0).alias('type'))

counts_ = sms_spam2_df.groupBy('type').count().collect()
spam_count = counts_[0][1]
ham_count = counts_[1][1]

In [4]:
sms_spam2_df.show()

+----+--------------------+
|type|                text|
+----+--------------------+
|   0|Go until jurong p...|
|   0|Ok lar... Joking ...|
|   1|Free entry in 2 a...|
|   0|U dun say so earl...|
|   0|Nah I don't think...|
|   1|FreeMsg Hey there...|
|   0|Even my brother i...|
|   0|As per your reque...|
|   1|WINNER!! As a val...|
|   1|Had your mobile 1...|
|   0|I'm gonna be home...|
|   1|SIX chances to wi...|
|   1|URGENT! You have ...|
|   0|I've been searchi...|
|   0|I HAVE A DATE ON ...|
|   1|XXXMobileMovieClu...|
|   0|Oh k...i'm watchi...|
|   0|Eh u remember how...|
|   0|Fine if that's th...|
|   1|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [5]:
# check here
sms_spam2_df.show()

+----+--------------------+
|type|                text|
+----+--------------------+
|   0|Go until jurong p...|
|   0|Ok lar... Joking ...|
|   1|Free entry in 2 a...|
|   0|U dun say so earl...|
|   0|Nah I don't think...|
|   1|FreeMsg Hey there...|
|   0|Even my brother i...|
|   0|As per your reque...|
|   1|WINNER!! As a val...|
|   1|Had your mobile 1...|
|   0|I'm gonna be home...|
|   1|SIX chances to wi...|
|   1|URGENT! You have ...|
|   0|I've been searchi...|
|   0|I HAVE A DATE ON ...|
|   1|XXXMobileMovieClu...|
|   0|Oh k...i'm watchi...|
|   0|Eh u remember how...|
|   0|Fine if that's th...|
|   1|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [6]:
# (5 pts)
np.testing.assert_array_equal(
    sms_spam2_df.groupBy('type').count().orderBy('type').rdd.map(lambda x: x['count']).collect(),
    [4827, 747])
np.testing.assert_array_equal(spam_count, 747)
np.testing.assert_array_equal(ham_count, 4827)

# Question 2.2: tfidf feature engineering
Create a pipeline that combines a `Tokenizer`, `CounterVectorizer`, and a `IDF` estimator to compute the tfidf vectors of each SMS. Fit this pipeline and assign the pipeline transformer to a variable `tfidf_pipeline`. The `Tokenizer` step should create a column `words`, the `CounterVectorizer` step should create a column `tf`, and the `IDF` step should create a column `tfidf`.

In [32]:
# create a Pipeline transformer and name it tfidf_pipeline
from pyspark.ml.feature import Tokenizer, CountVectorizer, IDF

# create Pipeline 
tfidf_pipeline = Pipeline(stages=[
    
    Tokenizer(inputCol='text',outputCol='words'),
    
    CountVectorizer(inputCol='words', outputCol='tf'),
    
    IDF(inputCol='tf', outputCol='tfidf')        
])

# check variable type, convey stages
print('object without fit:\n',
      type(tfidf_pipeline),
      '\n',
      tfidf_pipeline.getStages())

# fits pipeline to entire dataset
tfidf_pipeline = tfidf_pipeline.fit(sms_spam2_df)

# check variable type, convey stages
print('\nObject after fit:\n',
      type(tfidf_pipeline),
      '\n',
      tfidf_pipeline.stages) # Vocabulary size: 13525

object without fit:
 <class 'pyspark.ml.pipeline.Pipeline'> 
 [Tokenizer_db1565a346b5, CountVectorizer_de3e9a1239a5, IDF_7663f4e3223a]

Object after fit:
 <class 'pyspark.ml.pipeline.PipelineModel'> 
 [Tokenizer_db1565a346b5, CountVectorizerModel: uid=CountVectorizer_de3e9a1239a5, vocabularySize=13525, IDFModel: uid=IDF_7663f4e3223a, numDocs=5574, numFeatures=13525]


In [8]:
# Check pipeline result
tfidf_pipeline.transform(sms_spam2_df).show()

+----+--------------------+--------------------+--------------------+--------------------+
|type|                text|               words|                  tf|               tfidf|
+----+--------------------+--------------------+--------------------+--------------------+
|   0|Go until jurong p...|[go, until, juron...|(13525,[8,42,51,6...|(13525,[8,42,51,6...|
|   0|Ok lar... Joking ...|[ok, lar..., joki...|(13525,[5,74,404,...|(13525,[5,74,404,...|
|   1|Free entry in 2 a...|[free, entry, in,...|(13525,[0,3,8,20,...|(13525,[0,3,8,20,...|
|   0|U dun say so earl...|[u, dun, say, so,...|(13525,[5,22,60,1...|(13525,[5,22,60,1...|
|   0|Nah I don't think...|[nah, i, don't, t...|(13525,[0,1,66,86...|(13525,[0,1,66,86...|
|   1|FreeMsg Hey there...|[freemsg, hey, th...|(13525,[0,2,6,10,...|(13525,[0,2,6,10,...|
|   0|Even my brother i...|[even, my, brothe...|(13525,[0,7,9,13,...|(13525,[0,7,9,13,...|
|   0|As per your reque...|[as, per, your, r...|(13525,[0,10,11,4...|(13525,[0,10,11,4...|

In [9]:
# (5 pts)
np.testing.assert_array_equal([type(s) for s in tfidf_pipeline.stages],
                              [feature.Tokenizer, feature.CountVectorizerModel, feature.IDFModel])

# Question 2.3: uppercase feature

Typical spam messages contain words that are upper case. Create a dataframe `sms_spam3_df` where you add a new column `has_uppercase` which contains an integer `1` if the first sequence of uppercase letters is longer or equal to 3 and an integer `0` otherwise. You can extract sequence of 3 or more uppercase letters by using the regular expression `[A-Z]{3,}`. You will use the function `fn.regexp_extract` to find those sequences and extract the first one (e.g., with index 0) and then use `fn.length` to compute the length of such sequence.

In [10]:
# create sms_spam3_df below
# YOUR CODE HERE

sms_spam3_df = sms_spam2_df.withColumn('has_uppercase', \
                                       when(sms_spam2_df.text.rlike('[A-Z]{3,}'),1).otherwise(0))

sms_spam3_df.show()

+----+--------------------+-------------+
|type|                text|has_uppercase|
+----+--------------------+-------------+
|   0|Go until jurong p...|            0|
|   0|Ok lar... Joking ...|            0|
|   1|Free entry in 2 a...|            0|
|   0|U dun say so earl...|            0|
|   0|Nah I don't think...|            0|
|   1|FreeMsg Hey there...|            0|
|   0|Even my brother i...|            0|
|   0|As per your reque...|            0|
|   1|WINNER!! As a val...|            1|
|   1|Had your mobile 1...|            1|
|   0|I'm gonna be home...|            0|
|   1|SIX chances to wi...|            1|
|   1|URGENT! You have ...|            1|
|   0|I've been searchi...|            0|
|   0|I HAVE A DATE ON ...|            1|
|   1|XXXMobileMovieClu...|            1|
|   0|Oh k...i'm watchi...|            0|
|   0|Eh u remember how...|            0|
|   0|Fine if that's th...|            0|
|   1|England v Macedon...|            1|
+----+--------------------+-------

The first three messages with `has_uppercase == 1` are as follows:

```python
sms_spam3_df.where('has_uppercase == 1').take(3)
```

```console
[Row(type=1, text='WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.', has_uppercase=1),
 Row(type=1, text='Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030', has_uppercase=1),
 Row(type=1, text='SIX chances to win CASH! From 100 to 20,000 pounds txt> CSH11 and send to 87575. Cost 150p/day, 6days, 16+ TsandCs apply Reply HL 4 info', has_uppercase=1)]
```

In [11]:
# try it here
sms_spam3_df.where('has_uppercase == 1').take(3)

[Row(type=1, text='WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.', has_uppercase=1),
 Row(type=1, text='Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030', has_uppercase=1),
 Row(type=1, text='SIX chances to win CASH! From 100 to 20,000 pounds txt> CSH11 and send to 87575. Cost 150p/day, 6days, 16+ TsandCs apply Reply HL 4 info', has_uppercase=1)]

In [12]:
# (5 pts)
np.testing.assert_equal(set(sms_spam3_df.columns), {'has_uppercase', 'text', 'type'})
np.testing.assert_equal(type(sms_spam3_df.schema['has_uppercase'].dataType), sql.types.IntegerType)
np.testing.assert_equal(sms_spam3_df.rdd.map(lambda x : x['has_uppercase']).sum(), 891)

# Question 2.4: Compare models

Using the following splits:

In [13]:
training_df, validation_df, testing_df = sms_spam2_df.randomSplit([0.6, 0.3, 0.1], seed=0)

In [14]:
[training_df.count(), validation_df.count(), testing_df.count()]

[3311, 1709, 554]

**(15 pts)** Create pipelines where the first stage is the `tfidf_pipeline` created above and the second stage is a `LogisticRegression` model with different regularization parameters ($\lambda$) and elastic net mixture ($\alpha$) as following:
$$\lambda = [0, 0.02, 0.1] $$
$$\alpha = [0.2, 0.4] $$

Try different combinations of $\lambda$ and $\alpha$ (6 combinations) to find out best parameters by using the area under the curve as the estimator. Fit those pipelines to the appropriate data split and assign the pipeline with the best model to a variable `best_model`. Also, assign $\lambda$ and $\alpha$ of the best model to `best_model_lambda` and `best_model_alpha`.

For example, the AUC on training of the first model is perfect:

```
evaluator.evaluate(lr_pipeline1.transform(training_df))
```

```console
1.0
```

In [42]:
# Testing all the combinations of alpha and lambda and assign the best one to `best_model` based on AUC.
# YOUR CODE HERE
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator 


# fix tfidf_pipeline so it is a Pipeline, not a PipelineModel fit with entire dataset
tfidf_pipeline = Pipeline(stages=[
    
    Tokenizer(inputCol='text',outputCol='words'),
    
    CountVectorizer(inputCol='words', outputCol='tf'),
    
    IDF(inputCol='tf', outputCol='tfidf')])


# make pipe function that inputs variable hyperparameters
def run_pipe(regParam, enp):
    pipe = Pipeline(stages=[
        tfidf_pipeline,
        LogisticRegression(labelCol='type', 
                           featuresCol='tfidf',
                           regParam= regParam, 
                           elasticNetParam=enp)])
    return pipe

# set hyperparameters
regParam = [0, .02, .1]
enp = [.2,.4]

# make list to store results
pipes = []

# init auc evaluator
evaluator = BinaryClassificationEvaluator()
evaluator.setLabelCol('type')

# iterate through hyperparameters and save their auc score in pipe
for l in regParam:
    for a in enp:
        pipe = run_pipe(l,a)
        model = pipe.fit(training_df)
        auc = evaluator.evaluate(model.transform(validation_df))
        pipes.append([('regParam',l),
                      ('elasticNetMix',a),
                      ('auc',auc)])

# sort list based on auc score high to low
pipes.sort(key = lambda x: x[2][1], reverse=True)
print('best model: ',pipes[0])

# get best model
best_model_lambda = pipes[0][0][1] 
best_model_alpha = pipes[0][1][1]
best_model = run_pipe(best_model_lambda, best_model_alpha).fit(training_df)

best model:  [('regParam', 0.02), ('elasticNetMix', 0.2), ('auc', 0.9900130378096487)]


In [43]:
print(best_model.stages[0].stages[1] ) # vocabulary size 9868


CountVectorizerModel: uid=CountVectorizer_d317088abd70, vocabularySize=9868


In [44]:
# (15 pts)
np.testing.assert_equal(type(best_model), pipeline.PipelineModel)

np.testing.assert_array_equal([type(s) for s in best_model.stages],
                              [pipeline.PipelineModel, classification.LogisticRegressionModel])

np.testing.assert_equal(best_model_lambda, 0.02)
np.testing.assert_equal(best_model_alpha, 0.2)

# Question 2.5: Generalization of best model

Using the right split and the best model selected before, compute the generalization performance and assign it to a variable `AUC_best`

In [45]:
# assign to AUC_best the AUC of the best model selected before
# YOUR CODE HERE

AUC_best = evaluator.evaluate(best_model.transform(testing_df))
AUC_best

0.982324286708421

In [46]:
# (5 pts)
np.testing.assert_approx_equal(AUC_best, 0.9883924843423813, significant=2)

Using the same split and the best model, compute and assign `precision`, `recall` and `f1_score`. You should first count the numbers in the confusion matrix, and then compute these metrics based on the formula.

In [74]:
# YOUR CODE HERE
from pyspark.mllib.evaluation import MulticlassMetrics

# compute metrics
predictionsAndLabels = best_model.transform(testing_df).select('prediction','type').rdd.map(lambda x: (x[0], float(x[1])))

metrics = MulticlassMetrics(predictionsAndLabels)

cm = metrics.confusionMatrix().toArray()

print(cm)

precision = metrics.precision(1)

recall = metrics.recall(1)

f1_score = metrics.fMeasure(1.0, 1.0)

print(precision, recall, f1_score)

[[479.   0.]
 [ 15.  60.]]
1.0 0.8 0.888888888888889


In [75]:
# (5 pts)
np.testing.assert_array_almost_equal([precision, recall, f1_score],
    [1.0, 0.7976190476190477, 0.8874172185430463], decimal=2)

# Question 2.6: Inference

Use the best pipeline fitted above (`best_model`) to create Pandas dataframes that contain the most negative words and the most positive words. In particular, create a dataframe `positive_words` with the columns `word` and `weight` with the top 20 positive words, sorted by descending coefficient. Similarly create a `negative_words` Pandas dataframe with the top 20 negative words where the coefficient are sorted in ascending order. **Hint: follow the `sentiment_analysis.ipynb` notebook in the repo**

In [85]:
# create positive_words and negative_words pandas dataframe below

vocabulary = best_model.stages[0].stages[1].vocabulary
weights    = best_model.stages[-1].coefficients.toArray()

coeffs_df = pd.DataFrame({'word': vocabulary, 'weight': weights})

positive_words = coeffs_df.sort_values(by=['weight'], ascending=False).head(20)
positive_words
negative_words = coeffs_df.sort_values(by=['weight'], ascending=True).head(20)

In [86]:
# examine positive vocabulary
positive_words.head()

,word,weight
4471,sexy?,0.687893
2692,widelive.com/index.,0.631896
8330,08714712388,0.539505
15,call,0.528276
164,won,0.502129


In [87]:
# examine solutions
negative_words.head()

,word,weight
1,i,-0.183502
3234,fightng,-0.077429
2389,dificult,-0.077429
3141,lose.,-0.077429
11,me,-0.068350


The `positive_words` and `negative_words` dataframe should look like this:

```python
positive_words.head()
```

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>word</th>
      <th>weight</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>7263</th>
      <td>sexy?</td>
      <td>0.642738</td>
    </tr>
    <tr>
      <th>3555</th>
      <td>widelive.com/index.</td>
      <td>0.588182</td>
    </tr>
    <tr>
      <th>15</th>
      <td>call</td>
      <td>0.537161</td>
    </tr>
    <tr>
      <th>12237		</th>
      <td>08714712388</td>
      <td>0.504090</td>
    </tr>
    <tr>
      <th>81</th>
      <td>txt</td>
      <td>0.495005</td>
    </tr>
  </tbody>
</table>

and 

```python
negative_words.head()
```

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>word</th>
      <th>weight</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>1</th>
      <td>i</td>
      <td>-0.183463</td>
    </tr>
    <tr>
      <th>3332</th>
      <td>lose.</td>
      <td>-0.074937</td>
    </tr>
    <tr>
      <th>3371</th>
      <td>fightng</td>
      <td>-0.074937</td>
    </tr>
    <tr>
      <th>3221</th>
      <td>dificult</td>
      <td>-0.074937</td>
    </tr>
    <tr>
      <th>13</th>
      <td>me</td>
      <td>-0.065904</td>
    </tr>
  </tbody>
</table>

In [88]:
# (5 pts)
np.testing.assert_equal(set(positive_words.columns), {'weight', 'word'})
np.testing.assert_equal(set(negative_words.columns), {'weight', 'word'})
np.testing.assert_approx_equal(positive_words.weight.sum(), 8.675741267346245, significant=1)
np.testing.assert_approx_equal(negative_words.weight.sum(), -0.7292131526997235, significant=1)
np.testing.assert_array_less(positive_words.weight.iloc[-1], positive_words.weight.iloc[0])
np.testing.assert_array_less(negative_words.weight.iloc[0], negative_words.weight.iloc[-1])

# Question 2.7
Use the dataframe `sms_spam3_df` to create a model where the first feature is `has_uppercase` and the next set of features are the tfidf of the text. Perform feature engineering in all features using a max absolute scaler ([`MaxAbsScaler`](https://spark.apache.org/docs/2.0.2/ml-features.html#maxabsscaler)). Do a logistic regression on the resulting scaled features with regularization parameter $\lambda = 0.2$ and elastic net mixture $\alpha=0.1$ for the entire data (all of `sms_spam3_df`). Since you have scaled all features to be within the same range, you can compare them. 

**(5 pts)** with code and comments, answer below

1. is `has_uppercase` a feature that is positively or negative related to an SMS being spam?
2. what is the ratio of the coefficient of `has_uppercase` to the biggest positive tfidf coefficient?

In [129]:
# your code and comments below
from pyspark.ml.feature import MaxAbsScaler

pipe2 = Pipeline(stages=[
    tfidf_pipeline,
    VectorAssembler(inputCols=['has_uppercase','tfidf'], outputCol='caseAndTFIDF'),
    MaxAbsScaler(inputCol='caseAndTFIDF',outputCol='features'),
    LogisticRegression(regParam=.2, elasticNetParam=.1, labelCol='type')
])

model2 = pipe2.fit(sms_spam3_df)

vocabulary2= model2.stages[0].stages[1].vocabulary
weights2   = model2.stages[-1].coefficients.toArray()[1:]
has_uppercase_weight = model2.stages[-1].coefficients[0]

coeffs2_df =  pd.DataFrame({'word': vocabulary2, 'weight': weights2})

spammy_words = coeffs2_df.sort_values(by=['weight'], ascending=False).head(20)
not_spammy_words = coeffs2_df.sort_values(by=['weight'], ascending=True).head(20)

print(spammy_words,'\nHas uppercase weight: ', has_uppercase_weight)

print('\nTop word vs. has_uppercase: ',has_uppercase_weight / spammy_words.reset_index().weight[0])

'''
1. the has_uppercase feature is positively associated with being spam.
2. the ratio of the has_uppercase weight compared to the biggest tfidf coefficient is .46, which means that the has_uppercase
    weight is less than half as strong of an indicator of spam compared to the presence of the word 'call'.
'''

         word    weight
15       call  2.011949
81        txt  1.580207
68       text  1.453053
96     mobile  1.439059
109     claim  1.378177
108         &  1.338472
167       win  1.317322
55       free  1.269747
95      reply  1.095526
101      stop  1.004069
0          to  0.917446
11       your  0.874955
233      chat  0.861721
178      cash  0.802828
448     tones  0.736775
153      now!  0.722393
275      tone  0.691631
288   receive  0.687651
541  delivery  0.687348
223   service  0.659367 
Has uppercase weight:  0.9289178747599827

Top word vs. has_uppercase:  0.4617005798337619


"\n1. the has_uppercase feature is positively associated with being spam.\n2. the ratio of the has_uppercase weight compared to the biggest tfidf coefficient is .46, which means that the has_uppercase\n    weight is less than half as strong of an indicator of spam compared to the presence of the word 'call'.\n"